In [ ]:
import os
import pandas as pd

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import dateutil.parser as dparser
from datetime import date

In [ ]:
# 公司財務資料
org_data = pd.read_excel('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/lpcr2comp2_all_data.xlsx')
org_data.head()

In [ ]:
data = os.listdir('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment')
len(data)

In [ ]:
# 日期轉換格式
for i in range(0, len(data)):
    df = pd.read_csv('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment/{}'\
                    .format(data[i]))
    
    df['date'] = df['date'].apply(lambda x:x.replace('Updated','') )
    df['date'] = df['date'].apply(lambda x:x.replace('ET','') )
    df['date'] = df['date'].apply(lambda x:x.replace('|      WSJ Pro','') )
    df['FacilityStartDate'] = None
    df['Facility_minus_day'] = None
    
    df = df[['date','sentiment_score_text','sentiment_text']]# 只留有需要的columns
    
    for j in range(0, df.shape[0]):
        # 加入事件發生日日期
        df['FacilityStartDate'] = dparser.parse(org_data[org_data['FacilityID']==int(data[i][0:6])].iloc[0]['FacilityStartDate'])
        
        try:
            df.loc[j,'date'] = dparser.parse(df.loc[j,'date'])   # dparser.parse()轉換日期格式
        except:
            df.loc[j,'date'] = dparser.parse(df.loc[j,'date'][7:])
        
        d1 = dparser.parse(str(df['FacilityStartDate'][0]))
        d0 = df['date'].iloc[j]
        d1 = dparser.parse(str(d1)[:-9])
        d0 = dparser.parse(str(d0)[:-9])
        delta = d1-d0
        df.loc[j,'Facility_minus_day'] = delta.days # delta.days計算事件發生前多少天
        
        df.to_csv('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment_final/{}'\
                 .format(data[i]), index=False)
        
    print('完成到:',i)

In [ ]:
# 合併相同日期的情緒分數
data1 = os.listdir('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment_final')
for i in range(0, len(data1)):
    df = pd.read_csv('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment_final/{}'\
                     .format(data1[i]))
    aggregate_function = {'sentiment_score_text':'mean'}
    df_date = df.groupby(['Facility_minus_day','FacilityStartDate']).aggregate(aggregate_function)
    df_date = df_date.reset_index()
    
    df_date.to_csv('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment_final/{}'\
                   .format(data1[i]),index=False)

In [ ]:
# 刪除某資料夾下的.DS_store
# os.system("/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment_final" -name .DS_Store | xargs rm -rf')

In [ ]:
# 將各檔案新聞分數整合成表格
data1 = os.listdir('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment_final')

a = list(a for a in range(181))
b = ['FacilityID','FacilityStartDate']
c = b+a
df_1 = pd.DataFrame(columns=c)

for i in range(0, len(data1)):
    df = pd.read_csv('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment_final/{}'\
                     .format(data1[i]))
    df = df.drop(columns=['FacilityStartDate'])
    df = df.sort_values(by=['Facility_minus_day'],ascending=True) # DataFrame排序
    df = df.set_index('Facility_minus_day')
    df = df.T
    # append格式(df_1)與資料(df)
    df_1 = df_1.append(df, ignore_index = True)
    
    df_1.loc[i,'FacilityID'] = data1[i][0:6]
    df_2 = pd.read_csv('/Users/sinjhanghuang/Documents/碩士畢業論文/柏凱聯合論文/柏凱合作文章2011-2015（整合版）/2011-2015 sentiment_final/{}'\
                       .format(data1[i]))
    df_1.loc[i,'FacilityStartDate'] = df_2['FacilityStartDate'][0]

In [ ]:
# 資料完成後調整用
cols = df_1.columns.tolist()
cols = cols[-2:] + cols[:-2]
df_1 = df_1[cols]
df_1 = df_1.drop(columns=0)
df_1

In [ ]:
# values都為nltk_sentiment原始分數
df_1.to_csv('sentiment_regression.csv', index=False)